#Data Load

In [1]:
!pip install lightgbm


[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import warnings
warnings.filterwarnings('ignore')

# 라이브러리

import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import VotingClassifier
import lightgbm as lgbm
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

In [3]:
#-*- coding:utf-8 -*-

In [4]:
!python --version
print('pandas', pd.__version__)
print('numpy', np.__version__)

Python 3.9.7
pandas 1.3.4
numpy 1.18.5


In [5]:
# Seed 고정

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) 

In [10]:
train = pd.read_csv('../train.csv',encoding = 'utf-8')
test = pd.read_csv('../test.csv',encoding = 'utf-8')
sub = pd.read_csv('../sample_submission.csv',encoding = 'utf-8')

# Feature Engineering

> 라벨인코딩

In [11]:
# qualitative to quantitative

qual_col = ['PRODUCT_CODE', 'LINE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i]) 
print('Done.')

Done.


In [12]:
train_x = train.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train['Y_Class']

test_x = test.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [13]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> 정규화

In [18]:
a=train.columns.str.contains('X')
train.coulumns[False]

array([False, False, False, ...,  True,  True,  True])

In [16]:
#.columns.str.contains("a") : a라는 문자가 들어있는 문자열 모두 찾음
x_col = train.columns[train.columns.str.contains('X')].tolist()
x_col

['X_1',
 'X_2',
 'X_3',
 'X_4',
 'X_5',
 'X_6',
 'X_7',
 'X_8',
 'X_9',
 'X_10',
 'X_11',
 'X_12',
 'X_13',
 'X_14',
 'X_15',
 'X_16',
 'X_17',
 'X_18',
 'X_19',
 'X_20',
 'X_21',
 'X_22',
 'X_23',
 'X_24',
 'X_25',
 'X_26',
 'X_27',
 'X_28',
 'X_29',
 'X_30',
 'X_31',
 'X_32',
 'X_33',
 'X_34',
 'X_35',
 'X_36',
 'X_37',
 'X_38',
 'X_39',
 'X_40',
 'X_41',
 'X_42',
 'X_43',
 'X_44',
 'X_45',
 'X_46',
 'X_47',
 'X_48',
 'X_49',
 'X_50',
 'X_51',
 'X_52',
 'X_53',
 'X_54',
 'X_55',
 'X_56',
 'X_57',
 'X_58',
 'X_59',
 'X_60',
 'X_61',
 'X_62',
 'X_63',
 'X_64',
 'X_65',
 'X_66',
 'X_67',
 'X_68',
 'X_69',
 'X_70',
 'X_71',
 'X_72',
 'X_73',
 'X_74',
 'X_75',
 'X_76',
 'X_77',
 'X_78',
 'X_79',
 'X_80',
 'X_81',
 'X_82',
 'X_83',
 'X_84',
 'X_85',
 'X_86',
 'X_87',
 'X_88',
 'X_89',
 'X_90',
 'X_91',
 'X_92',
 'X_93',
 'X_94',
 'X_95',
 'X_96',
 'X_97',
 'X_98',
 'X_99',
 'X_100',
 'X_101',
 'X_102',
 'X_103',
 'X_104',
 'X_105',
 'X_106',
 'X_107',
 'X_108',
 'X_109',
 'X_110',
 'X_111'

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x_col = train.columns[train.columns.str.contains('X')].tolist()
scaler = MinMaxScaler()
  
scaler.fit(train_x[x_col])
  
train_x[x_col] = scaler.transform(train_x[x_col])
test_x[x_col] = scaler.transform(test_x[x_col])
  
train_x.head()

> 결측값 'train_x' 평균으로 채우기

In [ ]:
train_x = train_x.fillna(train_x.mean())
test_x = test_x.fillna(train_x.mean())

> 전체 결측값(Null)인 X변수 Drop하기

In [ ]:
train_x = train_x.dropna(axis =1)
test_x = test_x.dropna(axis=1)

# Modeling

In [ ]:
train_x.shape, train_y.shape, test_x.shape

In [ ]:
#lightGBM, ExtraTreesClassifier, RandomForestClassifier 선택

model = lgbm.LGBMClassifier()
et_cls = ExtraTreesClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=2795)
rf_cls = RandomForestClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=2795)
lg_cls = model.fit(train_x, train_y)

# 모델 voting

voting = VotingClassifier(
    estimators=[
        ('et', et_cls),
        ('rf', rf_cls),
        ('lg', lg_cls)
    ]
)
voting.fit(train_x, train_y)

# Predict

In [ ]:
pred = voting.predict(test_x)
sub['Y_Class'] = pred

sub.to_csv('./answer.csv', index=False)